In [ ]:
# 해당 셀을 실행한 후에 반드시 "런타임 다시시작"을 해주세요
!pip3 install kfp google-cloud-aiplatform matplotlib --upgrade -q --user

In [ ]:
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, component, ClassificationMetrics, Metrics)
from kfp.v2.google import experimental

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform import hyperparameter_tuning as hpt

## GCP 연결

In [ ]:
# gcp 연결
from google.colab import auth as google_auth

google_auth.authenticate_user()

# 환경변수 설정
아래 항목들을 본인 환경에 맞게 수정해주세요.

* PROJECT_ID = <프로젝트 ID>
* REGION = <리전>
* BUCKET_NAME = <bucket 이름>
* USER = <user 이름>

In [ ]:
from datetime import datetime

PROJECT_ID = 'gold-doodad-326906'
REGION = "us-central1"

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "gs://post-summarizer"

USER = "JeongMin-Do"
PIPELINE_ROOT = "{}/pipeline_root/{}".format(BUCKET_NAME, USER)
vertex_ai.init(project=PROJECT_ID)

# Parameter 지정

In [ ]:
# create working dir to pass to job spec
import time

ts = int(time.time())
WORKING_DIR = f"{PIPELINE_ROOT}/{ts}"

MODEL_DISPLAY_NAME = f"train_deploy{ts}"
print(WORKING_DIR)
print(MODEL_DISPLAY_NAME)

gs://post-summarizer/pipeline_root/JeongMin-Do/1632672294
train_deploy1632672294


# hpo job 정의

#### 해당 부분을 요구에 맞게 수정해주세요

In [ ]:
worker_pool_specs = [{
    "machine_spec": {
        "machine_type": "n1-standard-4",
        "accelerator_type": "NVIDIA_TESLA_T4",
        "accelerator_count": 1
    },
    "replica_count": 1,
    "container_spec": {
        "image_uri": "silverstar456/vertexai:hpo"
    }
}]

metric_spec={'accuracy':'maximize'}

parameter_spec = {
    "learning_rate": hpt.DoubleParameterSpec(min=0.001, max=1, scale="log"),
    "momentum": hpt.DoubleParameterSpec(min=0, max=1, scale="linear"),
    "num_neurons": hpt.DiscreteParameterSpec(values=[64, 128, 512], scale=None)
}

my_custom_job = vertex_ai.CustomJob(display_name='horses-humans-sdk-job',
                              worker_pool_specs=worker_pool_specs,
                              staging_bucket=BUCKET_NAME)

hp_job = vertex_ai.HyperparameterTuningJob(
    display_name='horses-humans-sdk-job',
    custom_job=my_custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=15,
    parallel_trial_count=3)

hp_job.run()